# Location of Model Artifacts

### Google Cloud Storage

- **model for inference** (965 MB): `https://storage.googleapis.com/issue_label_bot/model/lang_model/models_22zkdqlr/trained_model_22zkdqlr.pkl`


- **encoder (for fine-tuning w/a classifier)** (965 MB): 
`https://storage.googleapis.com/issue_label_bot/model/lang_model/models_22zkdqlr/trained_model_encoder_22zkdqlr.pth`


- **fastai.databunch** (27.1 GB):
`https://storage.googleapis.com/issue_label_bot/model/lang_model/data_save.pkl`


- **checkpointed model** (2.29 GB): 
`https://storage.googleapis.com/issue_label_bot/model/lang_model/models_22zkdqlr/best_22zkdqlr.pth`

### Weights & Biases Run

`https://app.wandb.ai/github/issues_lang_model/runs/22zkdqlr/overview`

# Part 1: Load Full Model + DataBunch In Order To Save Model For Inference

A fastai learner comes packaged with the training data and other data, however for inference we don't need this.  There is a way to export just the model weights without the data with `learn.export` or just the encoder base with `learn.save_encoder`.  Unfortunately, I forgot to do this during model training therefore we need to load the full checkpointed model and a databunch and save these artificacts for inference.  

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"


from pathlib import Path
from fastai.basic_train import load_learner
from fastai.text import load_data
from fastai.text.learner import language_model_learner
from fastai.text.models import AWD_LSTM, awd_lstm_lm_config

emb_sz=800
qrnn=False
bidir=False
n_layers=4
n_hid=2400


# https://app.wandb.ai/github/issues_lang_model/runs/22zkdqlr/overview
data_path = Path('/ds/lang_model')
model_path = data_path/'models_22zkdqlr'


def pass_through(x):
    return x


awd_lstm_lm_config.update(dict(emb_sz=emb_sz, qrnn=qrnn, bidir=bidir, n_layers=n_layers, n_hid=n_hid))

## Load learner object


Note: you don't have to do this over and over again, you just have to call `learn.export()` to save the learner after you have loaded everything.

In [2]:
data_lm = load_data(data_path, bs=96)

learn = language_model_learner(data=data_lm,
                               arch=AWD_LSTM,
                               model_dir=model_path,
                               pretrained=False)

#### Load weights of trained model

In [3]:
learn.load('best_22zkdqlr')

LanguageLearner(data=TextLMDataBunch;

Train: LabelList (16762799 items)
x: LMTextList
xxbos xxxfldtitle xxmaj grab excerpt and image using xxmaj open xxmaj graph xxxfldbody xxmaj we can update the wordpress excerpt and featured image by using xxmaj open xxmaj graph to get that information . xxmaj idea comes from xxmaj dan xxmaj scott via e - mail .,xxbos xxxfldtitle xxmaj gracefully handling ctrl+c ignores xxmaj program xxmaj main exit code . xxxfldbody xxmaj repro : 
  xxxlistb dotnet new console xxmaj paste the repro code below into xxmaj program.cs dotnet run xxmaj hit xxmaj ctrl + c xxmaj look at the last exit code . xxmaj we expect in the below repo for the exit code to be 1337 xxmaj built a repro that 's inspired by xxup asp.net 's hosting bits here : https : xxxfilepath xxmaj repro : using xxmaj system ; using xxmaj system . xxmaj threading ; using xxmaj system . xxmaj threading . xxmaj tasks ; namespace xxunk xxxjson 
  / cc xxxatmention xxxatmention xxxliste,xxbos xxxfldtitle

## Export Minimal Model State For Inference

In [4]:
learn.export('trained_model_22zkdqlr.pkl')

In [ ]:
learn.save_encoder('trained_model_encoder_22zkdqlr')

The data is very large so if you are running this notebook best to release memory by deleting these objects and loading the more lightweight inference artifacts that we just saved.

In [6]:
del learn
del data_lm

# Part II:  Load Minimal Model For Inference

In [7]:
from inference import InferenceWrapper, pass_through

#### Create an `InferenceWrapper` object

In [9]:
wrapper = InferenceWrapper(model_path='/ds/lang_model/models_22zkdqlr/',
                           )

In [10]:
issue_string = '# hello abacadabra world \nA second line **something bold**.'

In [11]:
pooledfeat = wrapper.get_pooled_features(issue_string)
print(pooledfeat)
print(pooledfeat.shape)

tensor([[-0.0168,  0.0424,  0.0784,  ...,  0.0782, -0.0466,  0.0152]],
       device='cuda:0', grad_fn=<CatBackward>)
torch.Size([1, 2400])


In [12]:
rawfeat = wrapper.get_raw_features(issue_string)
print(rawfeat)
print(rawfeat.shape)

tensor([[[-0.0038, -0.0458,  0.0113,  ...,  0.0395,  0.0149,  0.0079],
         [ 0.0141, -0.0070,  0.0614,  ...,  0.0283,  0.0589, -0.0534],
         [-0.0280,  0.0892,  0.0803,  ..., -0.0164,  0.0138,  0.1296],
         ...,
         [ 0.0004,  0.1302,  0.1215,  ..., -0.0525,  0.1411, -0.0619],
         [-0.0224,  0.1757,  0.1024,  ...,  0.0663,  0.0730,  0.1068],
         [ 0.0479, -0.0142,  0.1249,  ...,  0.0782, -0.0466,  0.0152]]],
       device='cuda:0', grad_fn=<CudnnRnnBackward>)
torch.Size([1, 12, 800])


#### Predict the next  5 words

We don't actually use this functionality, but it is interesting to see for those who are curious what the output of a langauge model is.  Recall that we are using the encoder of the language model to extract features from GitHub issues.

In [19]:
wrapper.learn.predict('I am having trouble opening a', 5)

'I am having trouble opening a project with the ios binary'